In [ ]:
import os
from ollama import Client

client = Client(
    host="https://ollama.com",
    headers={'Authorization': 'Bearer ___'}
)

messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
]

for part in client.chat('gpt-oss:120b', messages=messages, stream=True):
  print(part['message']['content'], end='', flush=True)

In [ ]:
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

# from smolagents import LiteLLMModel

# model = LiteLLMModel(
#         model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
#         api_base="http://127.0.0.1:11434",  # Default Ollama local server
#         num_ctx=8192,
#     )

In [6]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

from smolagents import LiteLLMModel

# model = LiteLLMModel(
#     model_id="ollama_chat/qwen2.5-coder:0.5b", # Much faster than 7b
#     api_base="http://127.0.0.1:11434"
# )
# 1. Setup the Gemma3:1b model
model = LiteLLMModel(
    model_id="ollama_chat/gemma3:1b", 
    api_base="http://127.0.0.1:11434",
    num_ctx=8192
)

In [ ]:
# Create a list of messages
messages = [
    {"role": "user", "content": [{"type": "text", "text": "Write a 5-line poem about coffee. in markdown format"}]}
]

# Call the model directly
response = model(messages)

# Print the text result
print(response.content)

In [14]:
@tool
def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

print(calculator.to_string())

Tool Name: calculator, Description: Multiply two integers., Arguments: a: int, b: int, Outputs: int


In [1]:
from huggingface_hub import login
login()

In [ ]:
from smolagents import CodeAgent, tool, InferenceClientModel

# Tool to suggest a menu based on the occasion
@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(tools=[suggest_menu], model=InferenceClientModel())

# Preparing the menu for the party
agent.run("Prepare a formal menu for the party.")

In [ ]:
from smolagents import CodeAgent, InferenceClientModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=InferenceClientModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

In [3]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, InferenceClientModel, Tool, tool, VisitWebpageTool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.
    
    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }
    
    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)
    
    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""
    
    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }
    
    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }
        
        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(), 
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool(),
	FinalAnswerTool()
    ], 
    model=InferenceClientModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me the best playlist for a party at the Wayne's mansion. want desi hip hop temhed rap songs like sedhe maut for refrnece, top 5 somgs only")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the best playlist for a party at the Wayne's mansion. want desi hip hop temhed rap songs like sedhe     │
│ maut for refrnece, top 5 somgs only                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Thinking ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to find the top 5 desi hip hop songs for a party at Wayne's mansion. I'll start by searching for   
"top 5 desi hip hop songs for party" using the web_search tool to get the latest and most relevant list.           
                                                                                                                   
<code>                                                                                                             
search_result = web_search("top 5 desi hip hop songs for party")                                                   
print(search_result)                                                                                               
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_result = web_search("top 5 desi hip hop songs for party")                                                 
  print(search_result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Desi Hip Hop Music Playlist: Best MP3 Songs on Gaana.com](https://gaana.com/playlist/gaana-dj-desi-hip-hop)
Desi Hip Hop playlist have 50 songs sung by Tsumyoki, MC Square, Yo Yo Honey Singh. ... Desi Hip Hop playlist songs
are in Hindi, Haryanvi, Punjabi ...

[2010s : Punjabi Hip Hop Music Playlist: Best 2010s : 
Punjabi](https://gaana.com/playlist/gaana-dj-2010s-punjabi-hip-hop)
2010s : Punjabi Hip Hop playlist have 23 songs sung by Bohemia, Sidhu Moose Wala, Sidhu Moose Wala. ... in music, 
offering over 30 million songs ...

[10 Albums to watch out for in this year - Desi Hip Hop](https://desihiphop.com/10-albums-watch-year-2016/180055)
New York, Los Angeles, Atlanta and even Miami, are all known as hotbeds for hip - hop , but over the past decade, 
Chicago has also emerged as a city ...

[RecentMusic - Latest Desi Hip Hop Releases](https://recentmusic.com/genre/desi-hip-hop/)
With its infectious energy, desi hip hop has become a significant force in shaping the sound of modern Indian and 
Pakistani music.

[Music - a compiled list | Dance Forums](https://www.dance-forums.com/threads/music-a-compiled-list.35117/)
... It Through the Grapevine " —Marvin Gaye Cha Cha Cha It s the Same Old Song (sung live by Rod Stewart) Cha Cha 
Cha My Lovin (En Vogue)? Hip Hop ...

[Hip Hop Archives - Artisfind](https://artisfind.com/genre/hip-hop/)
101.1 The Wiz – WIZF CINCINNATIS HOME FOR THE HIP - HOP COMMUNITY ... 101.9 Kiss FM – WIKS FOR HIP - HOP AND R& IN 
THE ENC!

[Hip Hop Archives • Project Radio](https://projectradio.net/genre/hip-hop/)
... from Zug, Switzerland, providing Urban Rap, Hip Hop ... Todays top 100 hits on 100Hitradio, Super concentrated 
for an insatiable hunger for top hits.

[Songlist](http://firstclassband.com/songlist/)
... Band Jump-Van Halen Everybody Everybody – Black Box Friends in Low Places – Garth Brooks Hip Hop Hooray – 
Naughty by Nature I Feel For ...

[DJ Shai Guy selects the biggest dance anthems of 
a](https://www.indiaweekly.biz/dj-shai-guy-selects-the-biggest-dance-anthems-of-a-party-fuelled-12-months/)
... of the UK’s leading club night promoters, Bombay Funkadelic, to select the most upbeat songs from this year 
ahead of their New Year’s Eve party ...

[Music — Blog — DJ Bash Homepage](https://bashnoise.com/blog/tag/Music)
My Top 20 Electronic Dance music tracks for the month of Sep 2025. ... fortunately the Tomorrowland team and 
countless amount people pulled the ...

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<code>                                                                                                             
top_5_songs = [                                                                                                    
    "1. ABCD by DIVINE, MC Altaf, Sammohit",                                                                       
    "2. Mat Kar Lala by Emiway Bantai",                                                                            
    "3. Hajamat (Red Bull 64 Bars) by Panther, Stunnah Beatz",                                                     
    "4. MUDDA by YUNG SAMMY, Wamp, Seedhe Maut",                                                                   
    "5. BEEN ON by Frappe Ash, toorjo dey, Encore ABJ"                                                             
]                                                                                                                  
final_answer(top_5_songs)                                                                                          
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  top_5_songs = [                                                                                                  
      "1. ABCD by DIVINE, MC Altaf, Sammohit",                                                                     
      "2. Mat Kar Lala by Emiway Bantai",                                                                          
      "3. Hajamat (Red Bull 64 Bars) by Panther, Stunnah Beatz",                                                   
      "4. MUDDA by YUNG SAMMY, Wamp, Seedhe Maut",                                                                 
      "5. BEEN ON by Frappe Ash, toorjo dey, Encore ABJ"                                                           
  ]                                                                                                                
  final_answer(top_5_songs)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: ['1. ABCD by DIVINE, MC Altaf, Sammohit', '2. Mat Kar Lala by Emiway Bantai', '3. Hajamat (Red Bull 
64 Bars) by Panther, Stunnah Beatz', '4. MUDDA by YUNG SAMMY, Wamp, Seedhe Maut', '5. BEEN ON by Frappe Ash, toorjo
dey, Encore ABJ']

[Step 3: Duration 8.70 seconds| Input tokens: 17,938 | Output tokens: 7,129]

['1. ABCD by DIVINE, MC Altaf, Sammohit',
 '2. Mat Kar Lala by Emiway Bantai',
 '3. Hajamat (Red Bull 64 Bars) by Panther, Stunnah Beatz',
 '4. MUDDA by YUNG SAMMY, Wamp, Seedhe Maut',
 '5. BEEN ON by Frappe Ash, toorjo dey, Encore ABJ']

In [ ]:
agent.push_to_hub('hijakaria/testMenuAgent')

In [ ]:
# Change to your username and repo name
alfred_agent = agent.from_hub('hijakaria/testMenuAgent', trust_remote_code=True)

alfred_agent.run("Prepare a chill menu for the party.")  

In [4]:
agent.push_to_hub('hijakaria/alfred_agent_v0')

README.md:   0%|          | 0.00/277 [00:00<?, ?B/s]

C:\Users\Jakaria.Ahmed\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jakaria.Ahmed\.cache\huggingface\hub\spaces--hijakaria--alfred_agent_v0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\Jakaria.Ahmed\AppData\Roaming\Python\Python312\site-packages\huggingface_h

This agent has step_callbacks: they will be ignored by this method. 1

CommitInfo(commit_url='https://huggingface.co/spaces/hijakaria/alfred_agent_v0/commit/5a86d8ffe542c6dba65138d982867cadcac398eb', commit_message='Upload agent', commit_description='', oid='5a86d8ffe542c6dba65138d982867cadcac398eb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/hijakaria/alfred_agent_v0', endpoint='https://huggingface.co', repo_type='space', repo_id='hijakaria/alfred_agent_v0'), pr_revision=None, pr_num=None)

In [ ]:
from smolagents import load_tool, CodeAgent, InferenceClientModel

image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

agent = CodeAgent(
    tools=[image_generation_tool],
    model=InferenceClientModel()
)

agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")